In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import f1_score
import lightgbm as lgb
from tpot import TPOTClassifier
from sklearn import preprocessing


def feature_generate_manually():
    train_path = '../data/hy_round1_train_20200102'
    test_path = '../data/hy_round1_testA_20200102'

    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    test_df_list = []
    for file_name in os.listdir(test_path):
        df = pd.read_csv(os.path.join(test_path, file_name))
        test_df_list.append(df)

    train_df = pd.concat(train_df_list)
    test_df = pd.concat(test_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
    test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df, test_df], sort=False)

    new_df = all_df.groupby('渔船ID').agg(x_min=('x', 'min'), x_max=('x', 'max'), x_mean=('x', 'mean'), x_std=('x', 'std'), x_skew=('x', 'skew'), x_sum=('x', 'sum'),
                y_min=('y', 'min'), y_max=('y', 'max'), y_mean=('y', 'mean'), y_std=('y', 'std'), y_skew=('y', 'skew'), y_sum=('y', 'sum'),
                v_min=('速度', 'min'), v_max=('速度', 'max'), v_mean=('速度', 'mean'), v_std=('速度', 'std'), v_skew=('速度', 'skew'), v_sum=('速度', 'sum'),
                d_min=('方向', 'min'), d_max=('方向', 'max'), d_mean=('方向', 'mean'), d_std=('方向', 'std'), d_skew=('方向', 'skew'), d_sum=('方向', 'sum'))
    new_df['x_max-x_min'] = new_df['x_max'] - new_df['x_min']
    new_df['y_max-y_min'] = new_df['y_max'] - new_df['y_min']
    new_df['x_max-y_min'] = new_df['x_max'] - new_df['y_min']
    new_df['y_max-x_min'] = new_df['y_max'] - new_df['x_min']

    new_df['slope'] = new_df['y_max-y_min'] / np.where(new_df['x_max-x_min']==0, 0.001, new_df['x_max-x_min'])
    new_df['area'] = new_df['x_max-x_min'] * new_df['y_max-y_min']

    new_df['type'] = all_df.groupby('渔船ID').agg(type=('type', 'first'))['type'].values

    X_train = new_df.drop(columns=['type']).iloc[:7000]
    y_train = new_df.iloc[:7000]['type']

    X_test = new_df.drop(columns=['type']).iloc[7000:]

    return X_train, y_train, X_test

def feature_generate_tsfresh_v1():
    train_df = pd.read_csv('./train.csv')
    X_train = train_df.drop(columns=['type'])
    y_train = train_df['type']

    test_df = pd.read_csv('./test.csv')
    X_test = test_df[X_train.columns]

    base_model = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)
    base_model.fit(X_train.values, y_train)

    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:24]]
    print(selected_columns)

    X_train = X_train[selected_columns]
    X_test = X_test[selected_columns]

    X_train_manully, _, X_test_manully = feature_generate_manually()

    X_train['x_max-x_min'] = X_train_manully['x_max-x_min'].values
    X_test['x_max-x_min'] = X_test_manully['x_max-x_min'].values
    X_train['x_max-y_min'] = X_train_manully['x_max-y_min'].values
    X_test['x_max-y_min'] = X_test_manully['x_max-y_min'].values
    X_train['y_max-x_min'] = X_train_manully['y_max-x_min'].values
    X_test['y_max-x_min'] = X_test_manully['y_max-x_min'].values
    X_train['y_max-y_min'] = X_train_manully['y_max-y_min'].values
    X_test['y_max-y_min'] = X_test_manully['y_max-y_min'].values

    X_train['slope'] = X_train_manully['slope'].values
    X_test['slope'] = X_test_manully['slope'].values
    X_train['area'] = X_train_manully['area'].values
    X_test['area'] = X_test_manully['area'].values

    for column in list(X_test.columns[X_test.isnull().sum() > 0]):
        mean_val = X_test[column].mean()
        X_test[column].fillna(mean_val, inplace=True)

    return X_train, y_train, X_test

train_path = '../data/hy_round1_train_20200102'
test_path = '../data/hy_round1_testA_20200102'

train_df_list = []
for file_name in os.listdir(train_path):
    df = pd.read_csv(os.path.join(train_path, file_name))
    train_df_list.append(df)

test_df_list = []
for file_name in os.listdir(test_path):
    df = pd.read_csv(os.path.join(test_path, file_name))
    test_df_list.append(df)

train_df = pd.concat(train_df_list)
test_df = pd.concat(test_df_list)

train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

all_df = pd.concat([train_df, test_df], sort=False)

def feature_generate_tsfresh():
    train_df = pd.read_csv('./train_v3.csv', index_col=['id'])
    X_train = train_df.drop(columns=['type'])

    y_train = []
    for ship_id, group in all_df.groupby('渔船ID'):
        if int(ship_id) >= 7000:
            break
        if not (int(ship_id) in [1709, 3423, 6605, 6791]):
            y_train.append(group['type'].values[0])
    y_train = np.array(y_train)

    test_df = pd.read_csv('./test_v3.csv', index_col=['id'])
    X_test = test_df[X_train.columns]

    print(X_train.shape, y_train.shape, X_test.shape)

    base_model = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)
    base_model.fit(X_train.values, y_train)

    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:24]]
    print(selected_columns)

    X_train = X_train[selected_columns]
    X_test = X_test[selected_columns]

    for column in list(X_test.columns[X_test.isnull().sum() > 0]):
        mean_val = X_test[column].mean()
        X_test[column].fillna(mean_val, inplace=True)

    return X_train, y_train, X_test

In [2]:
X_train_v1, _, X_test_v1 = feature_generate_tsfresh_v1()
X_train_v2, y_train, X_test_v2 = feature_generate_tsfresh()

X_train = np.concatenate((X_train_v2.values, X_train_v1.iloc[X_train_v2.index].values), axis=1)
X_test = np.concatenate((X_test_v2.values, X_test_v1.values), axis=1)


le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

Index(['x__quantile__q_0.1', 'x__minimum', 'y__quantile__q_0.9', 'y__maximum',
       'x__quantile__q_0.2', 'y__quantile__q_0.8', 'y__quantile__q_0.7',
       '速度__number_crossing_m__m_1', 'y__minimum', 'x__quantile__q_0.3',
       '速度__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'y__number_cwt_peaks__n_1', 'x__maximum', 'x__quantile__q_0.9',
       '速度__quantile__q_0.7', 'x__quantile__q_0.4',
       '方向__ar_coefficient__k_10__coeff_0', 'y__quantile__q_0.6',
       '速度__fft_coefficient__coeff_6__attr_"real"',
       '方向__fft_coefficient__coeff_64__attr_"abs"',
       '速度__ratio_beyond_r_sigma__r_2',
       'x__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       'y__fft_coefficient__coeff_6__attr_"angle"',
       '速度__agg_autocorrelation__f_agg_"mean"__maxlag_40'],
      dtype='object')
(6996, 962) (6996,) (2000, 962)
Index(['x__minimum', 'y__maximum', 'x__quantile__q_0.1', 'y__quantile__q_0.9',
       'y__quantile__q_0.8', 'y__minimum', 'x__quantile__q_0.2',
    

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [4]:
def get_model():
    exported_pipeline = make_pipeline(
        RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.7000000000000001, n_estimators=100), step=0.1),
        StandardScaler(),
        StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=0.01, fit_intercept=False, l1_ratio=1.0, learning_rate="invscaling", loss="perceptron", penalty="elasticnet", power_t=0.5)),
        GradientBoostingClassifier(learning_rate=0.5, max_depth=7, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=2, n_estimators=100, subsample=0.8500000000000001)
    )
    set_param_recursive(exported_pipeline.steps, 'random_state', 2020)
    return exported_pipeline

In [5]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [6]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=2020, shuffle=True)

model_v1_list = []
score_v1_list = []
for train_index, test_index in kf.split(X_train):
    model_v1 = get_model()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v1.fit(X_train[train_index], y_train[train_index])
    model_v1_list.append(model_v1)
    score_v1_list.append(f1_score(y_train[test_index], model_v1.predict(X_train[test_index]), average='macro'))
    
print(score_v1_list)
print(np.mean(score_v1_list), np.std(score_v1_list))

[0.9164502064457349, 0.9050152762658424, 0.8958189132787983, 0.9275599270020992, 0.9102222890814685]
0.9110133224147885 0.010684776007628294


In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

import os
import lightgbm as lgb
from sklearn import preprocessing

In [8]:
def get_model_v2():
    exported_pipeline = make_pipeline(
        SelectPercentile(score_func=f_classif, percentile=48),
        StackingEstimator(estimator=SGDClassifier(alpha=0.01, eta0=0.01, fit_intercept=False, l1_ratio=0.25, learning_rate="invscaling", loss="modified_huber", penalty="elasticnet", power_t=10.0)),
        ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.6000000000000001, min_samples_leaf=1, min_samples_split=3, n_estimators=100)
    )

    set_param_recursive(exported_pipeline.steps, 'random_state', 2020)
    return exported_pipeline

In [12]:
result_list = []
for model in model_v1_list:
    result = model.predict_proba(X_test)
    result_list.append(result)

# result = np.argmax(np.sum(np.array(result_list), axis=0) / 5, axis=1)

# result = le.inverse_transform(result)
# pd.DataFrame(result, index=range(7000, 9000)).to_csv('result.csv', header=None)

In [10]:
def feature_generate_manually():
    train_path = '../data/hy_round1_train_20200102'
    test_path = '../data/hy_round1_testA_20200102'

    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    test_df_list = []
    for file_name in os.listdir(test_path):
        df = pd.read_csv(os.path.join(test_path, file_name))
        test_df_list.append(df)

    train_df = pd.concat(train_df_list)
    test_df = pd.concat(test_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
    test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df, test_df], sort=False)

    new_df = all_df.groupby('渔船ID').agg(x_min=('x', 'min'), x_max=('x', 'max'), x_mean=('x', 'mean'), x_std=('x', 'std'), x_skew=('x', 'skew'), x_sum=('x', 'sum'),
                y_min=('y', 'min'), y_max=('y', 'max'), y_mean=('y', 'mean'), y_std=('y', 'std'), y_skew=('y', 'skew'), y_sum=('y', 'sum'),
                v_min=('速度', 'min'), v_max=('速度', 'max'), v_mean=('速度', 'mean'), v_std=('速度', 'std'), v_skew=('速度', 'skew'), v_sum=('速度', 'sum'),
                d_min=('方向', 'min'), d_max=('方向', 'max'), d_mean=('方向', 'mean'), d_std=('方向', 'std'), d_skew=('方向', 'skew'), d_sum=('方向', 'sum'))
    new_df['x_max-x_min'] = new_df['x_max'] - new_df['x_min']
    new_df['y_max-y_min'] = new_df['y_max'] - new_df['y_min']
    new_df['x_max-y_min'] = new_df['x_max'] - new_df['y_min']
    new_df['y_max-x_min'] = new_df['y_max'] - new_df['x_min']

    new_df['slope'] = new_df['y_max-y_min'] / np.where(new_df['x_max-x_min']==0, 0.001, new_df['x_max-x_min'])
    new_df['area'] = new_df['x_max-x_min'] * new_df['y_max-y_min']

    new_df['type'] = all_df.groupby('渔船ID').agg(type=('type', 'first'))['type'].values

    X_train = new_df.drop(columns=['type']).iloc[:7000]
    y_train = new_df.iloc[:7000]['type']

    X_test = new_df.drop(columns=['type']).iloc[7000:]

    return X_train, y_train, X_test


In [11]:
def feature_generate_tsfresh():
    train_df = pd.read_csv('./train.csv')
    X_train = train_df.drop(columns=['type'])
    y_train = train_df['type']

    test_df = pd.read_csv('./test.csv')
    X_test = test_df[X_train.columns]

    base_model = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)
    base_model.fit(X_train.values, y_train)

    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:24]]
    print(selected_columns)

    X_train = X_train[selected_columns]
    X_test = X_test[selected_columns]

    X_train_manully, _, X_test_manully = feature_generate_manually()

    X_train['x_max-x_min'] = X_train_manully['x_max-x_min'].values
    X_test['x_max-x_min'] = X_test_manully['x_max-x_min'].values
    X_train['x_max-y_min'] = X_train_manully['x_max-y_min'].values
    X_test['x_max-y_min'] = X_test_manully['x_max-y_min'].values
    X_train['y_max-x_min'] = X_train_manully['y_max-x_min'].values
    X_test['y_max-x_min'] = X_test_manully['y_max-x_min'].values
    X_train['y_max-y_min'] = X_train_manully['y_max-y_min'].values
    X_test['y_max-y_min'] = X_test_manully['y_max-y_min'].values

    X_train['slope'] = X_train_manully['slope'].values
    X_test['slope'] = X_test_manully['slope'].values
    X_train['area'] = X_train_manully['area'].values
    X_test['area'] = X_test_manully['area'].values
    
    for column in list(X_test.columns[X_test.isnull().sum() > 0]):
        mean_val = X_test[column].mean()
        X_test[column].fillna(mean_val, inplace=True)

    return X_train.values, y_train.values, X_test.values

In [13]:
X_train_tsfresh, y_train, X_test_tsfresh = feature_generate_tsfresh()

Index(['x__quantile__q_0.1', 'x__minimum', 'y__quantile__q_0.9', 'y__maximum',
       'x__quantile__q_0.2', 'y__quantile__q_0.8', 'y__quantile__q_0.7',
       '速度__number_crossing_m__m_1', 'y__minimum', 'x__quantile__q_0.3',
       '速度__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'y__number_cwt_peaks__n_1', 'x__maximum', 'x__quantile__q_0.9',
       '速度__quantile__q_0.7', 'x__quantile__q_0.4',
       '方向__ar_coefficient__k_10__coeff_0', 'y__quantile__q_0.6',
       '速度__fft_coefficient__coeff_6__attr_"real"',
       '方向__fft_coefficient__coeff_64__attr_"abs"',
       '速度__ratio_beyond_r_sigma__r_2',
       'x__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       'y__fft_coefficient__coeff_6__attr_"angle"',
       '速度__agg_autocorrelation__f_agg_"mean"__maxlag_40'],
      dtype='object')


In [14]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

X_train = np.concatenate([X_train_tsfresh], axis=1)
X_test = np.concatenate([X_test_tsfresh], axis=1)

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=2020, shuffle=True)

model_v2_list = []
score_v2_list = []
for train_index, test_index in kf.split(X_train):
    model_v2 = get_model_v2()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v2.fit(X_train[train_index], y_train[train_index])
    model_v2_list.append(model_v2)
    score_v2_list.append(f1_score(y_train[test_index], model_v2.predict(X_train[test_index]), average='macro'))
    
print(score_v2_list)
print(np.mean(score_v2_list), np.std(score_v2_list))

[0.9131358979146258, 0.9148225333896688, 0.9010780680648377, 0.9147134533985596, 0.918325967249146]
0.9124151840033676 0.005917167535311547


In [16]:
for model in model_v2_list:
    result = model.predict_proba(X_test)
    result_list.append(result)

In [17]:
result = np.argmax(np.sum(np.array(result_list), axis=0) / 10, axis=1)

result = le.inverse_transform(result)
pd.DataFrame(result, index=range(7000, 9000)).to_csv('result.csv', header=None)